In [11]:
import ezdxf
import numpy as np

In [52]:
# Create a new DXF document.
doc = ezdxf.new(dxfversion='R2010')

# Create new table entries (layers, linetypes, text styles, ...).
layer = doc.layers.new('main', dxfattribs={'lineweight': 0.0254})
layer.rgb = (255, 0, 0)

# DXF entities (LINE, TEXT, ...) reside in a layout (modelspace, 
# paperspace layout or block definition).  
msp = doc.modelspace()

In [53]:
# Define constants
WIDTH = 140 # mm
HEIGHT = 300 # mm
n_x = 14
n_y = 30
gap = 1.5 # mm
BUFFER_HEIGHT = 20 # mm, extra length on end to use as a handle
seamhole_diameter = 3 # mm

# Define derived constants
cell_width = WIDTH // n_x
cell_height = HEIGHT // n_y

In [54]:
# Define border
msp.add_line((0,  -BUFFER_HEIGHT), (WIDTH,  -BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((0,  -BUFFER_HEIGHT), (0, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((WIDTH,  -BUFFER_HEIGHT), (WIDTH, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((0, HEIGHT + BUFFER_HEIGHT), (WIDTH, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})

<class 'ezdxf.entities.line.Line'> LINE(#31)

In [55]:
# Define horizontal cuts
for j in range(1, n_y):
    if j % 2 == 1:
        for i in range(n_x // 2 + 1):
            start_pos = (max(0, cell_width*(2*i-1) + gap/2), cell_height*j)
            end_pos = (min(WIDTH, cell_width*(2*i+1) - gap/2), cell_height*j)
            msp.add_line(start_pos, end_pos)
    else:
        for i in range(n_x // 2):
            start_pos = (cell_width*(2*i) + gap/2, cell_height*j)
            end_pos = (cell_width*(2*i+2) - gap/2, cell_height*j)
            msp.add_line(start_pos, end_pos)

In [56]:
# Define vertical cuts
for i in range(1, n_x):
    if i % 2 == 0:
        for j in range(n_y // 2 + 1):
            start_pos = (cell_width*i, max(0, cell_height*(2*j-1) + gap/2))
            end_pos = (cell_width*i, min(HEIGHT, cell_height*(2*j+1) - gap/2))
            msp.add_line(start_pos, end_pos)
    else:
        for j in range(n_y // 2):
            start_pos = (cell_width*i, cell_height*(2*j) + gap/2)
            end_pos = (cell_width*i, cell_height*(2*j+2) - gap/2)
            msp.add_line(start_pos, end_pos)

In [57]:
num_points = 100
theta = np.arange(0, 2*np.pi, 2*np.pi/num_points)

In [58]:
# Define seam holes
for i in [cell_width/2, WIDTH - cell_width/2]:
    for j in [cell_height/2 + cell_height*j for j in range(n_y)]:
        for n in range(num_points - 1):
            start_pos = (i + seamhole_diameter*np.cos(theta[n])/2, j + seamhole_diameter*np.sin(theta[n])/2)
            end_pos = (i + seamhole_diameter*np.cos(theta[n+1])/2, j + seamhole_diameter*np.sin(theta[n+1])/2)
            msp.add_line(start_pos, end_pos)     
        start_pos = (i + seamhole_diameter*np.cos(theta[-1])/2, j + seamhole_diameter*np.sin(theta[-1])/2)
        end_pos = (i + seamhole_diameter*np.cos(theta[0])/2, j + seamhole_diameter*np.sin(theta[0])/2)
        msp.add_line(start_pos, end_pos)
#         msp.add_circle((i, j), radius=seamhole_diameter/2)

In [59]:
# Save DXF document.
doc.saveas('20201123_square_1cm_1.5mm_3mmseamhole_lined.dxf')